In [1]:
import os, sys
import numpy as np
import json
import neuroglancer
from matplotlib import pyplot as plt
viewer = neuroglancer.Viewer()
viewer

http://127.0.0.1:36867/v/fe0bb573ef21249085916fcd11d2d9e19ea554d7/

In [2]:
#color_filepath = os.path.join('./', 'neuroglancer/contours/json_cache', 'struct_to_color.json')
#with open(color_filepath, 'r') as json_file:
#    colors = json.load(json_file)
#print('Color maps:', colors)
HOME = os.path.expanduser("~")
DIR = os.path.join(HOME, 'programming/pipeline_utility')
sys.path.append(DIR)
from utilities.contour_utilities import get_structure_colors
colors = get_structure_colors()

Connecting dklab@192.168.1.12:3306


In [3]:
VOL_DIR = '/net/birdstore/Active_Atlas_Data/copied_from_S3/mousebrainatlas-data/CSHL_volumes/atlasV7/atlasV7_10.0um_scoreVolume/score_volumes'
files = os.listdir(VOL_DIR)
numpy_files = [f for f in files if f.endswith('.npy') and 'surround' not in f]
print('Number of numpy files:', len(numpy_files))

Number of numpy files: 51


## MD589 reference

In [ ]:
MD589_VOLUME_PATH = '/net/birdstore/Active_Atlas_Data/data_root/CSHL_volumes/MD589/MD589_full_filled.npy'
MD589 = np.load(MD589_VOLUME_PATH)
MD589 = np.swapaxes(MD589, 0, 2)
np.unique(MD589, return_counts=True)

In [ ]:
layer = neuroglancer.SegmentationLayer(
    source=neuroglancer.LocalVolume(
        data=MD589, 
        dimensions=neuroglancer.CoordinateSpace(names=['x', 'y', 'z'], units='um', scales=[5, 5, 20]), 
        voxel_offset=(0, 0, 0)
    ),
)
with viewer.txn() as s:
#     s.layers.clear()
    s.layers['layer'] = layer

## Aggregation

In [15]:
x_length = 2000
y_length = 2000
z_length = 600
full_brain_volume_annotated = np.zeros((z_length, y_length, x_length), dtype=np.uint8)

for n in numpy_files:
    prefix = os.path.splitext(n)[0]
    structure = prefix.split('_', 3)[3]
        
    try:
        color = colors[structure.upper()]
    except:
        sided = '{}_R'.format(structure.upper())
        color = colors[sided]

    volume_filename = os.path.join(VOL_DIR, n)
    volume_input = np.load(volume_filename)
    volume_input = np.swapaxes(volume_input, 0, 2)
    volume_input = np.swapaxes(volume_input, 1, 2)

    volume_nonzero_indices = volume_input > 0.9
    volume_input[volume_nonzero_indices] = color
    structure_volume = volume_input.astype(np.uint8)

    origin_filename = os.path.join(VOL_DIR, '{}_origin_wrt_canonicalAtlasSpace.txt'.format(prefix))
    origin_wrt = np.loadtxt(origin_filename)
    x,y,z = origin_wrt
    x_start = int(x) + x_length // 2
    y_start = int(y) + y_length // 2
    z_start = int(z) + z_length // 2
    x_end = x_start + structure_volume.shape[2]
    y_end = y_start + structure_volume.shape[1]
    z_end = z_start + structure_volume.shape[0]

    try:
        full_brain_volume_annotated[z_start:z_end, y_start:y_end,x_start:x_end] = structure_volume
    except:
        print('Error\t',str(structure).ljust(8), str(color).rjust(2), end="\t")
        print('shape', str(structure_volume.shape).rjust(18), end=", ")
        print('x range', str(x_start).rjust(4), str(x_end).rjust(4), end=", ")
        print('y range', str(y_start).rjust(4), str(y_end).rjust(4), end=", ")
        print('z range', str(z_start).rjust(4), str(z_end).rjust(4))
        
full_brain_volume_annotated = np.swapaxes(full_brain_volume_annotated, 0, 2)

Fit	 Tz_R     24	shape      (61, 47, 106), x range  878  984, y range 1096 1143, z range  319  380
Fit	 4N_L      4	shape       (21, 23, 19), x range  891  910, y range  887  910, z range  266  287
Fit	 PBG_R    14	shape       (34, 91, 67), x range  868  935, y range  827  918, z range  469  503
Fit	 Sp5C_L   21	shape    (182, 165, 167), x range 1191 1358, y range 1006 1171, z range   88  270
Fit	 RMC_L    16	shape       (87, 58, 91), x range  782  873, y range  913  971, z range  202  289
Fit	 LC_L     12	shape       (31, 71, 93), x range  954 1047, y range  912  983, z range  202  233
Fit	 Tz_L     24	shape      (61, 47, 106), x range  878  984, y range 1096 1143, z range  221  282
Fit	 LC_R     12	shape       (31, 71, 93), x range  954 1047, y range  912  983, z range  368  399
Fit	 SNC_R    19	shape     (104, 109, 80), x range  736  816, y range  922 1031, z range  367  471
Fit	 7n_L      7	shape    (150, 108, 107), x range  940 1047, y range  993 1101, z range  122  272
Fit	 VCA_R

In [16]:
test_layer = neuroglancer.SegmentationLayer(
    source = neuroglancer.LocalVolume(
        data=full_brain_volume_annotated, 
        dimensions=neuroglancer.CoordinateSpace(names=['x', 'y', 'z'], units='um', scales=[10, 10, 10]), 
        voxel_offset=(0, 0, 0)
    ),
)

with viewer.txn() as s:
    s.layers.clear()
    s.layers['aggregate'] = test_layer
print(viewer)

http://127.0.0.1:36867/v/fe0bb573ef21249085916fcd11d2d9e19ea554d7/


## Individual volume 

In [ ]:
for file in numpy_files:
    volume_filename = os.path.join(VOL_DIR, file)
    volume_input = np.load(volume_filename)
    volume_input[volume_input > 0.9] = 1
    volume_input = np.swapaxes(volume_input, 0, 2)
    
    dimensions = neuroglancer.CoordinateSpace(names=['x', 'y', 'z'], units='um', scales=[10, 10, 10])
    
    prefix = os.path.splitext(file)[0]
    origin_filename = os.path.join(VOL_DIR, '{}_origin_wrt_canonicalAtlasSpace.txt'.format(prefix))
    origin_wrt = np.loadtxt(origin_filename)
    
    volume_layer = neuroglancer.SegmentationLayer(
        source = neuroglancer.LocalVolume(data=volume_input, dimensions=dimensions, voxel_offset=[0, 0, 0]),
    )
    with viewer.txn() as s:
        s.layers[prefix.split('_', 3)[3]] = volume_layer